In [5]:
import pandas as pd
import openai
import json
import os
from datetime import datetime

# OpenAI API anahtarını çevresel değişkenden al
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print("OpenAI API Anahtarı başarıyla yüklendi.")
else:
    print("UYARI: OpenAI API Anahtarı bulunamadı. Lütfen ~/.zshrc dosyasını kontrol edin ve source edin.")

print(f"Pandas Versiyonu: {pd.__version__}")
print(f"OpenAI Versiyonu: {openai.__version__}")

# JSON dosyasını oku
try:
    # Mevcut çalışma dizinini kontrol edelim
    print(f"Mevcut Çalışma Dizini: {os.getcwd()}")

    # Dosyanın varlığını kontrol edelim
    file_path = 'cleaned_result.json'
    if not os.path.exists(file_path):
        print(f"Hata: '{file_path}' dosyası bulunamadı. Lütfen dosyanın '{os.getcwd()}' klasöründe olduğundan emin olun.")
    else:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        df = pd.DataFrame(data)
        print("Veri başarıyla yüklendi. İlk 5 satır:")
        print(df.head())
        print(f"Toplam kayıt sayısı: {len(df)}")
except json.JSONDecodeError:
    print("Hata: 'Result_38.json - yeni adıyla cleaned_result.json' dosyası geçerli bir JSON formatında değil.")
except Exception as e:
    print(f"Beklenmedik bir hata oluştu: {e}")

OpenAI API Anahtarı başarıyla yüklendi.
OpenAI API Anahtarı başarıyla yüklendi.
Pandas Versiyonu: 2.3.1
OpenAI Versiyonu: 1.97.1
Mevcut Çalışma Dizini: /Users/berkaybakac/Projects/lipyum_oto_karar
Veri başarıyla yüklendi. İlk 5 satır:
    İş ID            Lokasyon                    Sektör    İş Gönderim Saati  \
0  115607  Şanlıurfa Haliliye         Buzdolabı Servisi  2025-07-22 16:00:02   
1  115605     İstanbul Pendik  Bulaşık Makinesi Servisi  2025-07-22 15:58:56   
2  115603    Balıkesir Karesi             Klima Montajı  2025-07-22 15:28:35   
3  115600    İstanbul Avcılar         Buzdolabı Servisi  2025-07-22 15:10:34   
4  115599     İzmir Karşıyaka  Çamaşır Makinesi Servisi  2025-07-22 15:07:58   

     İade Talebi Saati          Hizmet Veren Geri Arama Saatleri  \
0  2025-07-22 16:32:59                       2025-07-22 16:30:46   
1  2025-07-22 17:27:46                       2025-07-22 17:26:54   
2  2025-07-22 15:38:12                       2025-07-22 15:33:56   
3  2025-07-2

In [11]:
# Bu hücre, daha kapsamlı veri temizliği, özellik mühendisliği ve İngilizce prompt hazırlığını içerir.

print("\n--- Veri Temizliği ve Ön İşleme Adımları Başlıyor ---")

# 1. Veri Tipi Dönüşümleri ve Boş Değer (NaN) İşlemleri
# Tarih-saat sütunlarını datetime formatına dönüştürme
date_cols = ["İş Gönderim Saati", "İade Talebi Saati", "Hizmet Veren Geri Arama Saatleri", "Partner Geri Arama Saatleri"]
for col in date_cols:
    if col in df.columns:
        # errors='coerce' geçersiz tarihleri NaT (Not a Time) yapar
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"'{col}' sütunu datetime formatına dönüştürüldü.")
    else:
        print(f"UYARI: '{col}' sütunu veri setinde bulunamadı. Tarih dönüşümü yapılmadı.")

# Metin sütunlarındaki boş/NaN değerleri boş stringe dönüştürme ve string tipine zorlama
# Bu, birleştirme işlemlerinde hata almamızı engeller
text_cols_to_clean = [
    "İade Talebi Nedeni", "Hizmet Veren Arama Metinleri", "Partner Arama Metinleri",
    "Partner Red Sebebi", "Müşteri Cevabı", "Sistem Cevabı", "Lokasyon", "Sektör"
]
for col in text_cols_to_clean:
    if col in df.columns:
        df[col] = df[col].fillna("").astype(str)
        print(f"'{col}' sütunundaki NaN değerler boş stringe dönüştürüldü ve str tipine zorlandı.")
    else:
        print(f"UYARI: '{col}' sütunu veri setinde bulunamadı. Metin temizliği yapılmadı.")


# 2. Tekrarlayan Verilerin Tespiti ve Giderilmesi (opsiyonel ama iyi bir pratik)
initial_rows = len(df)
df.drop_duplicates(inplace=True)
if len(df) < initial_rows:
    print(f"Tekrarlayan {initial_rows - len(df)} kayıt silindi. Yeni kayıt sayısı: {len(df)}")
else:
    print("Tekrarlayan kayıt bulunamadı.")


# 3. Aykırı Değerler veya Anlamsız Verilerin İlk Elemesi (Örnek: Çok kısa veya çok uzun metinler)
# Bu adım, domain bilgisi gerektirebilir. Şimdilik temel bir filtreleme yapalım.
# Örneğin, "İade Talebi Nedeni" çok kısaysa veya boşsa atlayabiliriz.
initial_rows_after_dedup = len(df)
df = df[df["İade Talebi Nedeni"].str.strip().astype(bool)].copy() # Boş veya sadece boşluk içerenleri sil
if len(df) < initial_rows_after_dedup:
    print(f"'İade Talebi Nedeni' boş olan {initial_rows_after_dedup - len(df)} kayıt silindi.")
else:
    print("'İade Talebi Nedeni' boş olan kayıt bulunamadı.")

# 4. Feature Engineering: İş Gönderim Saati ile İade Talebi Saati Arası Süre
print("\n--- Feature Engineering Adımları Başlıyor ---")
if "İş Gönderim Saati" in df.columns and "İade Talebi Saati" in df.columns:
    # Geçerli datetime değerleri olanları filtrele
    # Sadece hem gönderim hem de iade talebi saati olanlar için hesaplama yap
    valid_time_records = df.dropna(subset=["İş Gönderim Saati", "İade Talebi Saati"])
    
    # Süre farkını hesapla (timedelta cinsinden)
    time_diff = valid_time_records["İade Talebi Saati"] - valid_time_records["İş Gönderim Saati"]
    
    # Süreyi dakika cinsinden yeni bir sütuna ata
    df['Time_Elapsed_Minutes'] = time_diff.dt.total_seconds() / 60
    
    # Negatif süreleri (iade talebi, gönderimden önce olamaz) ve çok büyük/küçük uç değerleri ele
    # Örneğin, negatif süreleri NaN yapalım
    df.loc[df['Time_Elapsed_Minutes'] < 0, 'Time_Elapsed_Minutes'] = None
    
    # NaN değerleri, metne dönüştürürken "belirtilmemiş" gibi bir ifade kullanacağız.
    print("'Time_Elapsed_Minutes' (İş-İade Arası Süre) özelliği oluşturuldu.")
else:
    print("UYARI: 'İş Gönderim Saati' veya 'İade Talebi Saati' sütunları bulunamadığı için zaman farkı özelliği oluşturulamadı.")

# 5. Filtreleme: Sadece Belirli İade Durumları
print("\n--- Nihai Veri Filtreleme Adımları Başlıyor ---")
initial_rows_before_final_filter = len(df)
if "İade Durumu" in df.columns:
    # PDF'te "Personel Redetti" geçse de JSON'da "Personel Kabul Etmedi" var. JSON'a uyalım.
    df_final = df[df["İade Durumu"].isin(["Personel Kabul Etti", "Personel Kabul Etmedi"])].copy()
    print(f"'{initial_rows_before_final_filter}' kayıttan, 'İade Durumu' 'Personel Kabul Etti' veya 'Personel Kabul Etmedi' olan '{len(df_final)}' kayıt filtrelendi.")
else:
    print("UYARI: 'İade Durumu' sütunu bulunamadığı için nihai filtreleme yapılamadı. Tüm uygun kayıtlar kullanılacak.")
    df_final = df.copy() # Sütun yoksa filtreleme yapma
    
# İşlenen DataFrame'i df olarak güncelleyelim
df = df_final

print(f"\nTemizleme, Özellik Mühendisliği ve Nihai Filtreleme sonrası toplam kayıt sayısı: {len(df)}")
print("İşlem görmüş DataFrame'in ilk 5 satırı (güncel):")
print(df.head())


# --- 6. Fine-tuning Veri Setini Oluşturma (İngilizce Prompt ile) ---

print("\n--- Fine-tuning Veri Seti Oluşturma (JSONL) Başlıyor ---")

fine_tuning_data = []

# Sistem mesajı - modelin genel rolünü İngilizce tanımlarız
system_message_content = (
    "You are an AI assistant that evaluates refund requests from service providers and classifies them "
    "as 'Accepted by Personnel' or 'Rejected by Personnel'. Your task is to make a clear decision based on "
    "the 'Reason for Refund Request', 'Service Provider Call Transcripts', 'Partner Call Transcripts', "
    "and 'Time Elapsed Between Job Submission and Refund Request' information provided. "
    "You must only respond with 'Accepted by Personnel' or 'Rejected by Personnel'."
)

# Her bir filtrelenmiş satır için dönüştürme yap
for index, row in df.iterrows():
    user_content_parts = []

    # Reason for Refund Request (İade Talebi Nedeni)
    if pd.notna(row["İade Talebi Nedeni"]) and str(row["İade Talebi Nedeni"]).strip() != "":
        user_content_parts.append(f"Reason for Refund Request: {row['İade Talebi Nedeni']}")

    # Service Provider Call Transcripts (Hizmet Veren Arama Metinleri)
    if pd.notna(row["Hizmet Veren Arama Metinleri"]) and str(row["Hizmet Veren Arama Metinleri"]).strip() != "":
        # Multiple entries separated by ||
        provider_transcripts = str(row["Hizmet Veren Arama Metinleri"]).split('||')
        user_content_parts.append(f"Service Provider Call Transcripts: {' '.join(provider_transcripts)}")

    # Partner Call Transcripts (Partner Arama Metinleri)
    if pd.notna(row["Partner Arama Metinleri"]) and str(row["Partner Arama Metinleri"]).strip() != "": # 'Partner Arama Metinleri'
        # Multiple entries separated by ||
        partner_transcripts = str(row["Partner Arama Metinleri"]).split('||')
        user_content_parts.append(f"Partner Call Transcripts: {' '.join(partner_transcripts)}")

    # Feature Engineering'den gelen süreyi ekle
    if 'Time_Elapsed_Minutes' in row and pd.notna(row['Time_Elapsed_Minutes']):
        minutes = round(row['Time_Elapsed_Minutes'])
        user_content_parts.append(f"Time Elapsed Between Job Submission and Refund Request: {minutes} minutes.")
    else:
        user_content_parts.append("Time Elapsed Between Job Submission and Refund Request: Not specified.")

    # Eğer user content kısmı oluşmadıysa bu kaydı atla
    if not user_content_parts:
        continue

    user_message_content = "\n".join(user_content_parts)

    # Assistant mesajını oluştur (İade Durumu)
    assistant_message_content = None
    if pd.notna(row["İade Durumu"]):
        if row["İade Durumu"] == "Personel Kabul Etti":
            assistant_message_content = "Accepted by Personnel"
        elif row["İade Durumu"] == "Personel Kabul Etmedi": # JSON'daki ifadeye göre
            assistant_message_content = "Rejected by Personnel"
        else:
            # Filtreleme adımı sayesinde buraya gelmemeli, ama bir kontrol olsun
            continue

    if assistant_message_content is None:
        continue

    messages = [
        {"role": "system", "content": system_message_content},
        {"role": "user", "content": user_message_content},
        {"role": "assistant", "content": assistant_message_content}
    ]

    fine_tuning_data.append({"messages": messages})

# Dönüştürülmüş veriyi JSONL formatında kaydet
output_jsonl_path = 'fine_tuning_data.jsonl'
with open(output_jsonl_path, 'w', encoding='utf-8') as f:
    for entry in fine_tuning_data:
        f.write(json.dumps(entry, ensure_ascii=False) + '\n')

print(f"\nFine-tuning veri seti '{output_jsonl_path}' dosyasına başarıyla kaydedildi.")
print(f"Toplam dönüştürülen ve kaydedilen örnek sayısı: {len(fine_tuning_data)}")


--- Veri Temizliği ve Ön İşleme Adımları Başlıyor ---
'İş Gönderim Saati' sütunu datetime formatına dönüştürüldü.
'İade Talebi Saati' sütunu datetime formatına dönüştürüldü.
'Hizmet Veren Geri Arama Saatleri' sütunu datetime formatına dönüştürüldü.
'Partner Geri Arama Saatleri' sütunu datetime formatına dönüştürüldü.
'İade Talebi Nedeni' sütunundaki NaN değerler boş stringe dönüştürüldü ve str tipine zorlandı.
'Hizmet Veren Arama Metinleri' sütunundaki NaN değerler boş stringe dönüştürüldü ve str tipine zorlandı.
'Partner Arama Metinleri' sütunundaki NaN değerler boş stringe dönüştürüldü ve str tipine zorlandı.
'Partner Red Sebebi' sütunundaki NaN değerler boş stringe dönüştürüldü ve str tipine zorlandı.
'Müşteri Cevabı' sütunundaki NaN değerler boş stringe dönüştürüldü ve str tipine zorlandı.
'Sistem Cevabı' sütunundaki NaN değerler boş stringe dönüştürüldü ve str tipine zorlandı.
'Lokasyon' sütunundaki NaN değerler boş stringe dönüştürüldü ve str tipine zorlandı.
'Sektör' sütunundak